Jacci Ziebert

May 4, 2020

Final Project

My analysis was done in 4 parts, each labeled by a number:
    1. Python tweet collection and cleaning.
    2. R finding matching game titles in tweets with an output of a clean dataframe with game stats from another API
    3. R doing data analysis of tweets and board games data. Output is a few charts
    4. R writing executive summary, importing graphs.

In [15]:
import sys
import os
import jsonpickle
import tweepy
import json
import string
import csv
from pandas.io.json import json_normalize
import pandas as pd

This first part gathers board game tweets.

In [6]:
exec(open('C:/Users/jacci/Documents/DS 710/ds710spring2020finalproject/twitter_credentials.py').read())
auth = tweepy.AppAuthHandler(consumer_key=con_key, consumer_secret=con_secret)

#Setting up new api wrapper, using authentication only
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
#Error handling
if (not api):
    print ("Problem connecting to API")

# api.rate_limit_status()['resources']['search'] # to check how many queries we have left

#This is what we are searching for
searchQuery = '-is:retweet -#goldengeekaward -apartments -RT -kickstarter -#kickstarter (#boardgames OR #boardgame OR #gamenight OR #bgstats OR #eurogames OR #boardgaming OR #bggplay)\
                AND (played OR playing OR plays OR tonight OR "last night" OR session OR solo)' #1,329

maxTweets = 10000 #Maximum number of tweets we want to collect
tweetsPerQry = 100 #The twitter Search API allows up to 100 tweets per query
tweetCount = 0

with open('C:/Users/jacci/Documents/DS 710/ds710spring2020finalproject/boardgames_tweets_test.json', 'w') as f:

    #Tell the Cursor method that we want to use the Search API (api.search)
    #Also tell Cursor our query, and the maximum number of tweets to return
    for tweet in tweepy.Cursor(api.search,q=searchQuery, tweet_mode='extended').items(maxTweets):
        #Write the JSON format to the text file, and add one to the number of tweets we've collected
        f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
        tweetCount += 1
    #Display how many tweets we have collected
    print("Downloaded {0} tweets".format(tweetCount))

Downloaded 1552 tweets


Then I did some cleaning, i.e. removed hashtags, urls, common words, and collected hashtags.

In [17]:
filename = "C:/Users/jacci/Documents/DS 710/ds710spring2020finalproject/boardgames_tweets4302020.json"

def import_games_json(filename = filename): #import raw tweets and outputs a df with selected columns
    with open(filename, 'r', encoding='utf-8') as tweet_data:
          games_tweet = [json.loads(line) for line in tweet_data.readlines()]
#           game_df = pd.json_normalize(games_tweet)
          game_df = json_normalize(games_tweet)
    game_df_selected = game_df[['id', 'full_text', 'user.screen_name', 'user.followers_count',\
    'user.friends_count', 'entities.hashtags','retweet_count', 'favorite_count', 'favorited',\
    'retweeted']]
    return game_df_selected

def return_cleaned_tweet(row): # Remove common words, hashtags, urls
    clean_text = row['full_text'].split()
    clean_text = [word.lower() for word in clean_text]
    clean_text = [word for word in clean_text if word[0] != "#" and word[0:4] != "http" \
    and word not in ("played", "play", "played", "playing" "i", "it's", "i'll",\
     "tonight", "got", "marathon", "vip", "review", "podcast", "games", "today", "tonight")]
    # clean_text = [word[:-1] for word in clean_text if word[-1] in string.punctuation]
    clean_text = ' '.join(word for word in clean_text)
    return clean_text

def return_hashtags(row): # Find hashtags
    hashtags = row['full_text'].split()
    hashtags = [word[1:] for word in hashtags if word[0] == "#" and word not in ("#boardgames",\
    "#boardgame", "#bggplay", "#gamenight", "#bgstats", "#sologaming", "#boardgamegeek",\
    "#tabletopgames","#tabletopgaming", "#games", "#solo")]
    hashtags = ' '.join(word.lower() for word in hashtags)
    return hashtags

def export_to_csv(df):
    df.fillna("", inplace=True)
    df.to_csv("C:/Users/jacci/Documents/DS 710/ds710spring2020finalproject/cleaned_games_test.csv")

game_tweets = import_games_json()
game_tweets['clean.full.text'] = game_tweets.apply(return_cleaned_tweet, axis=1) # new column of cleaned tweet
game_tweets['hashtags'] = game_tweets.apply(return_hashtags, axis=1)  # new column of hashtags
# take all capitalized and hashtags, returns all lowercase
export_to_csv(game_tweets)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None): # i want to print EVERYTHING
#     print(game_tweets['clean.full.text'][1:5])